In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("longshort_strategy.ipynb")

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

def isclose(value, original, tolerance= 0.05):
    return value <= original * (1+tolerance) and value >= original * (1-tolerance)

In this homework, we will:
- Introduce pandas through data cleaning and processing.
- Implement a basic trading strategy.
- Model the costs of a trading strategy.
- Calculate expected returns.
- Use expected returns to create a trading strategy.
- Try to add predictive features/signals.

# Pandas intro

In [3]:
!pip install lxml

  Using cached lxml-5.3.2-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.6 kB)
Using cached lxml-5.3.2-cp311-cp311-manylinux_2_28_x86_64.whl (5.0 MB)


In [4]:
# list of sp500 companies, from wikipedia
url = 'https://en.m.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1219339331'

# pandas lets us read html tables from wikipedia page
data = pd.read_html(url)

You can add cells in the notebook to perform quick operations or tests. Note that the last value in a cell will automatically be displayed below the cell once it finishes running. Let's examine the type of variable that `data` returns.

In [5]:
type(data)

list

We can access values in a list using indexing and slicing operations:

- `list[i]` will return the \(i\)-th value in the list, starting with 0.
- `list[i:]` will return the \(i\)-th value and onward.
- `list[:i]` will return all values up to the \(i\)-th value.
- `list[i:j]` will return values from the \(i\)-th value onward, up to but not including the \(j\)-th value.

In [6]:
# looks like data returns a list of tables, what type is each table? 
type(data[0])

pandas.core.frame.DataFrame

In [7]:
# the dataframe is the core object of pandas, we'll abbreviate as df
# a dataframe is essentially a code version of a table
df = data[0]
df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [8]:
# we can look at the first n rows of a table using df.head(n)
# please do so here
df_head5 = df.head(5)

In [9]:
grader.check("q1a")

q1a results: All test cases passed!

In this analysis, we are not particularly interested in the columns for security, headquarters, date added, founded, or CIK values. We can select specific columns of a DataFrame by passing a list of column names:

```python
columns = ['colname1', 'colname2', ...]
df[columns]
```

Please select every column except for security, headquarters, CIK, founded, and date added.

In [10]:
columns = ["Symbol","GICS Sector", "GICS Sub-Industry"]
df[columns]

,Symbol,GICS Sector,GICS Sub-Industry
0,MMM,Industrials,Industrial Conglomerates
1,AOS,Industrials,Building Products
2,ABT,Health Care,Health Care Equipment
3,ABBV,Health Care,Biotechnology
4,ACN,Information Technology,IT Consulting & Other Services
...,...,...,...
498,XYL,Industrials,Industrial Machinery & Supplies & Components
499,YUM,Consumer Discretionary,Restaurants
500,ZBRA,Information Technology,Electronic Equipment & Instruments
501,ZBH,Health Care,Health Care Equipment


In [11]:
grader.check("q1b")

q1b results: All test cases passed!

It appears there are quite a few sectors! In a DataFrame, one column is referred to as a series. We can select a series using `df['colname']`. By using `series.unique()`, we can obtain a list of the unique values within a series.

Please return the number of unique values in the 'GICS Sector' column. Note that you can programmatically access the length of this array using `.size`.

In [12]:
unique_sectors = len(df["GICS Sector"].unique())
unique_sectors

11

In [13]:
grader.check("q1c")

q1c results: All test cases passed!

If we want to determine how many times each unique value appears in a series, we can use `series.value_counts()`. This function associates each unique value with the number of times it appears in the series.

Please return the number of times 'Information Technology' appears in the 'GICS Sector' column. Note that you can programmatically access the count for 'Information Technology' using `series.value_counts()['uniquevalname']`.

In [14]:
it_sectors = int(df["GICS Sector"].value_counts()["Information Technology"])

In [15]:
grader.check("q1d")

q1d results: All test cases passed!

One way to select values from a DataFrame is by using index-based location, or `iloc`. We can use `df.iloc[index for rows, index for columns]` to specify which indices to select.

- `df.iloc[:,:]` will return the entire DataFrame.
- `df.iloc[:5, :6]` will return the first 5 rows and 6 columns.
- `df.iloc[2:5, 3:]` will return the third column and onward, for rows 2-4.

Please return rows 5-9, and return the 2nd column and onward.

In [16]:
specified_slice = df.iloc[5:10, 2:]
specified_slice

,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
5,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
6,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
7,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981
8,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
9,Health Care,Life Sciences Tools & Services,"Santa Clara, California",2000-06-05,1090872,1999


In [17]:
grader.check("q1e")

q1e results: All test cases passed!

We can also examine the sub-industries, each of which is associated with one of the sectors. To select only the rows of the DataFrame that correspond to a specific sector, we can use boolean indexing.

A boolean operator can be created generally using `df['colname'] {comparator} {value}`. This returns a series of `False` and `True` values for each row.

Please create a boolean operator for when the 'GICS Sector' is 'Health Care' and return the number of `False` values using `value_counts()`.

In [18]:
false_healthcare = int((df["GICS Sector"] == "Health Care").value_counts()[False])
false_healthcare

439

In [19]:
grader.check("q1f")

q1f results: All test cases passed!

Now that we have our boolean operator, we can select the values that are `True`. We do this using `df[operator]`, where `operator = df['colname'] {comparator} {value}`.

Please use the operator you created to select only the rows where the security is in the 'Energy' sector (using the 'GICS Sector' column).

In [20]:
energy = df[df["GICS Sector"] == "Energy"]

In [21]:
grader.check("q1g")

q1g results: All test cases passed!

We also have `df.loc`, which works similarly to `df.iloc`. Here are some examples:

- `df.loc[row_indexer, column_indexer]`
- `df.loc[start_row_label:end_row_label, start_col_label:end_col_label]`
- `df.loc[list_of_row_labels, list_of_column_labels]`
- `df.loc[boolean_array_for_rows, boolean_array_for_columns]`

Please select the rows where the 'GICS Sector' is 'Industrials', and return the 1st through 4th columns.

In [22]:
first4cols_industrials = df[df["GICS Sector"] == "Industrials"].iloc[:, :4]

In [23]:
grader.check("q1h")

q1h results: All test cases passed!

Another useful DataFrame operation is sorting. We can sort an entire DataFrame using `df.sort_values('colname', ascending=True/False)`.

Please sort, in ascending order, the symbols in the 'Consumer Discretionary' sector, and report the 4th oldest security (by date added). You may want to break this into three separate tasks to make it easier.

In [24]:
old_4_security = (df[df["GICS Sector"] == "Consumer Discretionary"].sort_values("Date added", ascending=True)).iloc[3,1]
old_4_security

'Bath & Body Works, Inc.'

In [25]:
grader.check("q1i")

q1i results: All test cases passed!

Let's also introduce the `groupby` operator. We can group a DataFrame based on a given column, such as the 'GICS Sector' column. After grouping a DataFrame, we commonly apply some kind of aggregation, such as a count, an average, etc. For example, we might use `df.groupby('colname').count()`.

Please return the average CIK number of the 'Real Estate' sector using the `df.groupby('colname').mean(numeric_only=True)` aggregation. You can also access a specific series by adding `df.groupby('colname1').mean(numeric_only=True)['colname2']` to view the aggregation of a particular series.

In [26]:
mean_realestate_cik = df[df["GICS Sector"] == "Real Estate"].groupby("GICS Sector").mean(numeric_only=True)["CIK"].reset_index().iloc[0,1]
mean_realestate_cik

np.float64(956436.7741935484)

In [27]:
grader.check("q1j")

q1j results: All test cases passed!

In [28]:
def get_common_subindustry(df):
    # .index returns the index of a series, in this case the unique values
    # since value counts is sorted in descending order
    # .value_counts().index[0] will give us the largest avlue
    
    # we can also use display to show the intermediate steps
    # we use series.values to access the list of values in the series
    
    print('-'*35)
    print(df['GICS Sector'].values[0])
    print('-'*35)
    display(df['GICS Sub-Industry'].value_counts())
    return df['GICS Sub-Industry'].value_counts().index[0]

In [29]:
df['GICS Sub-Industry'].value_counts().index[0]

'Health Care Equipment'

In [30]:
df.groupby('GICS Sector').apply(get_common_subindustry)

-----------------------------------
Communication Services
-----------------------------------


GICS Sub-Industry
Interactive Media & Services             4
Movies & Entertainment                   4
Broadcasting                             3
Cable & Satellite                        2
Integrated Telecommunication Services    2
Interactive Home Entertainment           2
Advertising                              2
Publishing                               2
Wireless Telecommunication Services      1
Name: count, dtype: int64

-----------------------------------
Consumer Discretionary
-----------------------------------


GICS Sub-Industry
Hotels, Resorts & Cruise Lines         8
Restaurants                            6
Homebuilding                           4
Casinos & Gaming                       4
Apparel, Accessories & Luxury Goods    4
Broadline Retail                       3
Other Specialty Retail                 3
Distributors                           3
Automobile Manufacturers               3
Automotive Retail                      3
Apparel Retail                         2
Home Improvement Retail                2
Automotive Parts & Equipment           2
Computer & Electronics Retail          1
Footwear                               1
Consumer Electronics                   1
Leisure Products                       1
Home Furnishings                       1
Name: count, dtype: int64

-----------------------------------
Consumer Staples
-----------------------------------


...

-----------------------------------
Real Estate
-----------------------------------


GICS Sub-Industry
Multi-Family Residential REITs     6
Retail REITs                       5
Telecom Tower REITs                3
Health Care REITs                  3
Office REITs                       2
Data Center REITs                  2
Real Estate Services               2
Self-Storage REITs                 2
Hotel & Resort REITs               2
Single-Family Residential REITs    1
Other Specialized REITs            1
Industrial REITs                   1
Timber REITs                       1
Name: count, dtype: int64

-----------------------------------
Utilities
-----------------------------------


GICS Sub-Industry
Electric Utilities                              14
Multi-Utilities                                 12
Independent Power Producers & Energy Traders     2
Water Utilities                                  1
Gas Utilities                                    1
Name: count, dtype: int64

GICS Sector
Communication Services                    Interactive Media & Services
Consumer Discretionary                  Hotels, Resorts & Cruise Lines
Consumer Staples                                Packaged Foods & Meats
Energy                              Oil & Gas Exploration & Production
Financials                            Asset Management & Custody Banks
Health Care                                      Health Care Equipment
Industrials               Industrial Machinery & Supplies & Components
Information Technology                                  Semiconductors
Materials                                          Specialty Chemicals
Real Estate                             Multi-Family Residential REITs
Utilities                                           Electric Utilities
dtype: object

# data downloading/cleaning

In [31]:
# the yfinance library provides access to lots of information about tickers available on yahoo finance
!pip install yfinance

  Using cached yfinance-0.2.55-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.9-cp311-cp311-linux_x86_64.whl
Using cached yfinance-0.2.55-py2.py3-none-any.whl (109 kB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)


In [32]:
import yfinance as yf

**Errors are expected as data is missing for some tickers, see the future cells!**

In [43]:
# we can download historical price data for these assets using
# yf.download(tickers)
# there's other optional parameters that we won't cover here
# lets get the list of symbols/tickers we care about, and then download them using .unique() on our dataframe
# convert the series to a list before you pass it to yf
# this may take a minute or two
# please put the data into a dataframe called hdf, or historical dataframe for short

tickers = list(df['Symbol'].unique())
hdf = yf.download(tickers, start='2023-10-01', end='2024-03-29')
hdf = hdf[hdf.index <= '2024-03-19']

[*********************100%***********************]  503 of 503 completed

8 Failed downloads:
['BRK.B', 'WRK', 'PXD', 'CTLT', 'MRO']: YFTzMissingError('possibly delisted; no timezone found')
['SOLV', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-10-01 -> 2024-03-29) (Yahoo error = "Data doesn\'t exist for startDate = 1696132800, endDate = 1711684800")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-10-01 -> 2024-03-29)')


In [44]:
grader.check("q2a")

q2a results: All test cases passed!

Let's take a look at the data we've just downloaded

In [45]:
hdf

Price      Adj Close                                       Close         ...  \
Ticker          BF.B BRK.B CTLT GEV MRO PXD SOLV WRK           A    AAL  ...   
Date                                                                     ...   
2023-10-02       NaN   NaN  NaN NaN NaN NaN  NaN NaN  109.716743  12.75  ...   
2023-10-03       NaN   NaN  NaN NaN NaN NaN  NaN NaN  109.083572  12.29  ...   
2023-10-04       NaN   NaN  NaN NaN NaN NaN  NaN NaN  110.478531  12.73  ...   
2023-10-05       NaN   NaN  NaN NaN NaN NaN  NaN NaN  109.172607  12.85  ...   
2023-10-06       NaN   NaN  NaN NaN NaN NaN  NaN NaN  109.459526  12.76  ...   
...              ...   ...  ...  ..  ..  ..  ...  ..         ...    ...  ...   
2024-03-13       NaN   NaN  NaN NaN NaN NaN  NaN NaN  146.490189  14.00  ...   
2024-03-14       NaN   NaN  NaN NaN NaN NaN  NaN NaN  143.903656  13.99  ...   
2024-03-15       NaN   NaN  NaN NaN NaN NaN  NaN NaN  146.153229  13.96  ...   
2024-03-18       NaN   NaN  NaN NaN NaN NaN  NaN NaN  144.775757  14.11  ...   
2024-03-19       NaN   NaN  NaN NaN NaN NaN  NaN NaN  146.064041  13.93  ...   

Price       Volume                                                          \
Ticker         WTW       WY     WYNN       XEL       XOM      XYL      YUM   
Date                                                                         
2023-10-02  726200  5795100  1712300   4875400  14395000  1978100  1965400   
2023-10-03  540400  4921800  1315300   5161400  13297100  1309800  2100400   
2023-10-04  749100  3954000  2289300   3948700  19622200  1614400  1381200   
2023-10-05  855000  3724500  1513600   3835400  20128800  1131000  1240200   
2023-10-06  897700  4318300  2470600   6337200  25363400  1149700  2111900   
...            ...      ...      ...       ...       ...      ...      ...   
2024-03-13  530100  2508500  1138400   5420000  17122200   872500  3361400   
2024-03-14  482900  4124900  1882700  11860000  22121000  1335500  3091900   
2024-03-15  885400  6848900  4404600  11153200  38263900  1537600  4164800   
2024-03-18  402800  3373100  1528700   7217700  16770500   720000  2384500   
2024-03-19  579000  2496400  1444100   5656800  14878800   806900  2266500   

Price                                 
Ticker          ZBH    ZBRA      ZTS  
Date                                  
2023-10-02  1445400  246000  1270200  
2023-10-03  1850400  313300  1671800  
2023-10-04  1597600  487500  1673400  
2023-10-05  1048200  448400  2083200  
2023-10-06  1570500  384000  1490600  
...             ...     ...      ...  
2024-03-13   847000  286200  5947400  
2024-03-14  1203900  375300  6432600  
2024-03-15  2672400  539300  3399500  
2024-03-18   972300  329300  4955000  
2024-03-19   949300  306000  3624000  

[117 rows x 2523 columns]

In [25]:
# looks like the index is the date, and we have multiple levels of columns!
# we can get the top level of columns using df.columns.get_level_values(i)
# let's look at the unique top level columns, with .unique()

In [46]:
hdf.columns.get_level_values(0).unique()

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

In [47]:
# the first level of columns has the 'adjusted close', or the closing price for that day, with interest rate adjusted
# we are also given 'volume', or the amount of shares traded that day
# the other stuff might be nice to have, but we'll just stick with adjusted close and volume for simplicity
# let's now select those
# to not destroy our old dataframe, we'll make a copy called hdf_raw with df.copy()
# once you make the copy, make hdf = hdf[columns]
hdf_raw = hdf.copy()
hdf = hdf[['Adj Close', 'Volume']]

2b or not 2b, the answer is not because I deleted the question :) Also, **the next cell is filled in for you, so there's nothing you need to do here.**

In [48]:
# you may have seen the following error message upon download: 

# 2 Failed downloads:
# - BF.B: No data found for this date range, symbol may be delisted
# - BRK.B: No data found, symbol may be delisted
# - GEV and SOLV are also recent additions without any data

# please use df.loc to view the data for these symbols
# you should return all rows, and use columns.get_level_values(1) to get the second level of columns, with the symbols
# note that you can use '.isin(list)' as a boolean operator to return if a value is in a list
# your code might look like

# df.loc[:,df.columns.get_level_values(1).isin([list of values to check])]
broken_df = hdf.loc[:,hdf.columns.get_level_values(1).isin(['BRK.B', 'BF.B', 'GEV', 'SOLV', 'PXD', 'WRK', 'CTLT', 'MRO'])]
broken_df

Price      Adj Close                                 Volume                 \
Ticker          BF.B BRK.B CTLT GEV MRO PXD SOLV WRK   BF.B BRK.B CTLT GEV   
Date                                                                         
2023-10-02       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2023-10-03       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2023-10-04       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2023-10-05       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2023-10-06       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
...              ...   ...  ...  ..  ..  ..  ...  ..    ...   ...  ...  ..   
2024-03-13       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2024-03-14       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2024-03-15       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2024-03-18       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   
2024-03-19       NaN   NaN  NaN NaN NaN NaN  NaN NaN    NaN   NaN  NaN NaN   

Price                        
Ticker     MRO PXD SOLV WRK  
Date                         
2023-10-02 NaN NaN  NaN NaN  
2023-10-03 NaN NaN  NaN NaN  
2023-10-04 NaN NaN  NaN NaN  
2023-10-05 NaN NaN  NaN NaN  
2023-10-06 NaN NaN  NaN NaN  
...         ..  ..  ...  ..  
2024-03-13 NaN NaN  NaN NaN  
2024-03-14 NaN NaN  NaN NaN  
2024-03-15 NaN NaN  NaN NaN  
2024-03-18 NaN NaN  NaN NaN  
2024-03-19 NaN NaN  NaN NaN  

[117 rows x 16 columns]

In [49]:
grader.check("q2c")

q2c results: All test cases passed!

In [56]:
# awesome, now lets select everything except for that, since we dont want to use this data
# we can invert a boolean indexer by using ~, eg. df.loc[:, ~<condition>]
# let hdf be equal to this new selection, and then drop nan values using df.dropna()
hdf = hdf.loc[: , ~hdf.columns.get_level_values(1).isin(['BRK.B', 'BF.B', 'GEV', 'SOLV', 'PXD', 'WRK', 'CTLT', 'MRO'])]
hdf = hdf.dropna()

In [57]:
grader.check("q2d")

q2d results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Please also check gradescope for any written assignments for this week.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)